# Sequential acquisition

In [2]:
import torch
import sys
import malt

In [3]:
data = malt.data.collections.esol()

Processing dgl graphs from scratch...
Processing molecule 1000/1128


Make model

In [4]:
model = malt.models.supervised_model.GaussianProcessSupervisedModel(
    representation=malt.models.representation.DGLRepresentation(
        out_features=128,
    ),
    regressor=malt.models.regressor.ExactGaussianProcessRegressor(
        in_features=128, out_features=2,
    ),
    likelihood=malt.models.likelihood.HeteroschedasticGaussianLikelihood(),
)


In [5]:
merchant = malt.agents.merchant.DatasetMerchant(data)
assayer = malt.agents.assayer.DatasetAssayer(data)

In [6]:
player = malt.agents.player.SequentialModelBasedPlayer(
    model=model,
    policy=malt.policy.Greedy(),
    trainer=malt.trainer.get_default_trainer(),
    merchant=merchant,
    assayer=assayer,
)

In [214]:
malt.policy.Greedy??

Init signature:
malt.policy.Greedy(
    utility_function=<function expected_improvement at 0x000001A66D78D0D8>,
    acquisition_size: int = 1,
)
Source:        
class Greedy(Policy):
    """ Greedy policy. """

    def __init__(
        self,
        utility_function=utility_functions.expected_improvement,
        acquisition_size: int = 1,
    ):
        super(Greedy, self).__init__()
        self.utility_function = utility_function
        self.acquisition_size = acquisition_size

    def forward(
        self, distribution: torch.distributions.Distribution
    ) -> torch.Tensor:
        score = self.utility_function(distribution)
        _, idxs = torch.topk(
            score,
            self.acquisition_size,
            dim=0,
        )
        return idxs
File:           c:\users\micha\dev\choderalab\malt\malt\policy.py
Type:           ABCMeta
Subclasses:     


In [21]:
data_loader = data.view(batch_size=len(data))
g, y = next(iter(data_loader))

distribution = model.condition(g)

In [266]:
torch.argmax(distribution.sample((len(data),)), axis=1).ravel()

tensor([164, 841, 436,  ..., 550,  90, 164])

In [215]:
%%time

pending_points = set()
while len(pending_points) < len(data):
    pending_point = torch.argmax(distribution.sample((1,))).item()
    pending_points.add(
        pending_point
    )

Wall time: 732 ms


In [213]:
torch.argmax(distribution.sample((1000,)), axis=1)

tensor([[ 741],
        [ 889],
        [ 482],
        [ 427],
        [1037],
        [ 913],
        [ 914],
        [  31],
        [ 707],
        [ 537],
        [ 560],
        [ 256],
        [ 828],
        [ 519],
        [ 759],
        [ 749],
        [ 515],
        [1063],
        [  88],
        [ 292],
        [ 548],
        [ 265],
        [ 356],
        [1107],
        [ 716],
        [ 353],
        [   1],
        [ 981],
        [ 942],
        [ 948],
        [1111],
        [ 832],
        [ 897],
        [1031],
        [ 668],
        [ 171],
        [ 940],
        [ 941],
        [ 508],
        [ 943],
        [ 465],
        [ 330],
        [ 503],
        [ 188],
        [ 757],
        [1111],
        [ 487],
        [1063],
        [ 732],
        [ 125],
        [ 962],
        [ 322],
        [ 216],
        [1045],
        [ 668],
        [ 601],
        [ 286],
        [ 752],
        [1045],
        [ 717],
        [ 600],
        [ 864],
        

In [170]:
len(pending_points)

100

In [82]:
# sample distribution until we have q unique points
def _f_X_argmax(distribution):
    thetas = distribution.sample((1,))
    argmax = thetas
f_X_argmax = lambda distribution: torch.argmax(, axis=1)
# thetas = 
# pending_pts = torch.argmax(thetas, axis=1)

In [110]:
f_X_argmax(distribution)

tensor([[929]])

In [31]:
torch.unique(pending_pts)

tensor([   0,    8,   11,   39,   40,   41,   46,   84,   88,  125,  142,  145,
         147,  150,  152,  163,  191,  192,  202,  225,  230,  231,  239,  243,
         257,  258,  282,  302,  304,  345,  357,  368,  375,  379,  382,  390,
         401,  405,  407,  446,  448,  457,  464,  471,  475,  477,  506,  544,
         566,  569,  573,  591,  604,  657,  664,  668,  672,  674,  688,  715,
         737,  750,  757,  771,  833,  842,  851,  853,  857,  861,  892,  902,
         907,  918,  922,  936,  952,  959,  962,  967,  980,  993,  996, 1001,
        1006, 1024, 1036, 1041, 1044, 1056, 1072, 1084, 1106, 1123])

In [ ]:
while True:
    if player.step() is None:
        break